In [40]:
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt

In [41]:
TrafikDataDF=pd.read_csv('İstanbulTraficData.csv',sep=';')
print("Veri Sayısı : "+str(TrafikDataDF["TrafficIndex"].count()))

Veri Sayısı : 43334


In [42]:
TrafikDataDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43334 entries, 0 to 43333
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TrafficIndex      43334 non-null  int64  
 1   TrafficIndexDate  43334 non-null  object 
 2   Date              43334 non-null  object 
 3   Time              43334 non-null  object 
 4   Day               43334 non-null  int64  
 5   Month             43334 non-null  int64  
 6   Year              43334 non-null  int64  
 7   Week              43334 non-null  int64  
 8   daysofweek        43236 non-null  float64
 9   day_name          43236 non-null  object 
 10  is_weekend        43236 non-null  object 
 11  Days_Of_Week      43334 non-null  int64  
 12  Day_Name          43334 non-null  object 
 13  Is_Weekend_Flag   43334 non-null  bool   
dtypes: bool(1), float64(1), int64(6), object(6)
memory usage: 4.3+ MB


In [43]:
#Eksik Gün Hesaplama
TrafikDataDF['TrafficIndexDate']=TrafikDataDF['TrafficIndexDate'].apply(pd.to_datetime)
Days=pd.to_datetime("today").date()-TrafikDataDF.TrafficIndexDate.dt.date.max()
Days=Days.days

In [44]:
#Eksik Saat Hesaplama
if (pd.to_datetime("today").time()==TrafikDataDF.loc[TrafikDataDF['TrafficIndexDate'].dt.date==TrafikDataDF.TrafficIndexDate.dt.date.max()].TrafficIndexDate.dt.time.max().strftime("%H:%M"))==False:
    Days=Days+1
    print("Değer Güncellendi. Yeni Değer : "+str(Days))

Değer Güncellendi. Yeni Değer : 1


In [45]:
#IBB veri setinden son 910 günün verisini 30 Dakika aralıklı olarak çekiyoruz
if Days!=0:
    response = requests.get("https://api.ibb.gov.tr/tkmservices/api/TrafficData/v1/TrafficIndexHistory/"+str(Days)+"/30M")
    TrafikDataDF=pd.concat([TrafikDataDF,pd.DataFrame(response.json())])
    print("Veri Güncellendi. Yeni Veri Sayısı : "+str(TrafikDataDF["TrafficIndex"].count()))

Veri Güncellendi. Yeni Veri Sayısı : 43383


In [ ]:
TrafikDataDF['TrafficIndexDate']=TrafikDataDF['TrafficIndexDate'].apply(pd.to_datetime)

TrafikDataDF['Date'] = TrafikDataDF.TrafficIndexDate.dt.date
TrafikDataDF['Time'] = TrafikDataDF.TrafficIndexDate.dt.time
TrafikDataDF["Day"]=TrafikDataDF.TrafficIndexDate.dt.day
TrafikDataDF["Month"]=TrafikDataDF.TrafficIndexDate.dt.month
TrafikDataDF["Year"]=TrafikDataDF.TrafficIndexDate.dt.year
TrafikDataDF["Week"]=TrafikDataDF.TrafficIndexDate.dt.week
TrafikDataDF['Days_Of_Week'] = TrafikDataDF['TrafficIndexDate'].dt.dayofweek
TrafikDataDF["Day_Name"]=TrafikDataDF.TrafficIndexDate.dt.day_name()
TrafikDataDF['Is_Weekend_Flag'] = np.where(TrafikDataDF['Day_Name'].isin(['Sunday', 'Saturday']), "True","False")

In [47]:
TrafikDataDF.to_csv('İstanbulTraficData.csv',sep=';',index=False)